In [10]:
import cv2
import mediapipe as mp
import numpy as np
import random
import time

mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.5)

# Cargar la imagen del pájaro
bird_image_path = 'bird.png'  # Asegúrate de tener esta imagen en el directorio correcto
bird_image = cv2.imread(bird_image_path, cv2.IMREAD_UNCHANGED)

tubos = []
item = None
puntaje = 0
velocidad_juego = 5  # Velocidad inicial del juego
inicio_juego = time.time()
ultimo_item = 0  # Tiempo desde la última aparición del item
item_capturado = False
tiempo_captura_item = 0
duracion_efecto_item = 5  # Duración en segundos del efecto del item

# Función para crear un nuevo tubo
def crear_tubo(image_width):
    altura = random.randint(100, 300)  # Ajusta estos valores según tu ventana de juego
    espacio = 150  # Espacio entre los tubos superior e inferior
    return [image_width, altura, espacio, False]  # El cuarto elemento es para verificar si el tubo ya fue superado

# Función para mover y dibujar tubos y detectar colisiones
def actualizar_tubos(image, bird_position):
    global puntaje, velocidad_juego
    tubo_superado = False
    bird_x, bird_y = bird_position

    for tubo in tubos:
        tubo[0] -= velocidad_juego  # Mover tubo
        # Dibujar tubo superior
        cv2.rectangle(image, (int(tubo[0]), 0), (int(tubo[0]) + 50, int(tubo[1])), (0, 255, 0), -1)
        # Dibujar tubo inferior
        cv2.rectangle(image, (int(tubo[0]), int(tubo[1]) + int(tubo[2])), (int(tubo[0]) + 50, image.shape[0]), (0, 255, 0), -1)
        
        # Detectar colisión
        if bird_x + 25 > tubo[0] and bird_x - 25 < tubo[0] + 50:
            if bird_y - 25 < tubo[1] or bird_y + 25 > tubo[1] + tubo[2]:
                return True  # Colisión detectada

        # Verificar si el tubo fue superado
        if not tubo[3] and bird_x > tubo[0] + 50:
            tubo[3] = True
            tubo_superado = True

    # Eliminar tubos fuera de la pantalla
    tubos[:] = [tubo for tubo in tubos if tubo[0] > -50]

    # Aumentar puntaje y velocidad si se supera un tubo
    if tubo_superado:
        puntaje += 1
        velocidad_juego += 0.3  # Incrementar velocidad

    return False  # No hubo colisión

# Función para crear un item amarillo
def crear_item(image_width, image_height):
    y_pos = random.randint(50, image_height - 50)
    return [image_width, y_pos, False]  # x, y, capturado

# Función para mover y dibujar el item amarillo y ajustar la velocidad del juego
def actualizar_item(image, bird_position, current_time):
    global item, ultimo_item, item_capturado, tiempo_captura_item, velocidad_juego
    if item is None:
        if current_time - ultimo_item > 7:  # Generar un nuevo item cada 7 segundos
            item = crear_item(image.shape[1], image.shape[0])
            ultimo_item = current_time
    else:
        item[0] -= int(velocidad_juego * 1.5)  # Mover el item más rápido
        cv2.circle(image, (int(item[0]), int(item[1])), 10, (0, 255, 255), -1)  # Dibujar item

        # Detectar captura del item
        bird_x, bird_y = bird_position
        if bird_x - 25 < item[0] < bird_x + 25 and bird_y - 25 < item[1] < bird_y + 25:
            item = None  # El pájaro capturó el item
            item_capturado = True
            tiempo_captura_item = current_time
            velocidad_juego *= 0.8  # Reducir velocidad

        if item and item[0] < 0:
            item = None  # El item salió de la pantalla

    # Restaurar la velocidad normal del juego después del efecto del item
    if item_capturado and current_time - tiempo_captura_item > duracion_efecto_item:
        velocidad_juego /= 0.5
        item_capturado = False

# Iniciar la captura de video
cap = cv2.VideoCapture(0)

# Última posición conocida de la nariz
last_x_nose, last_y_nose = -1, -1

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignorando el frame vacío.")
        continue

    # Convertir la imagen de BGR a RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detectar el rostro
    results = face_detection.process(image_rgb)

    # Dibujar los resultados
    if results.detections:
        for detection in results.detections:
            # Calcular las coordenadas de la nariz
            nose_tip = detection.location_data.relative_keypoints[2]
            last_x_nose = int(nose_tip.x * image.shape[1])
            last_y_nose = int(nose_tip.y * image.shape[0])

    # Si se detectó la nariz, actualizar la posición del pájaro
    if last_x_nose != -1 and last_y_nose != -1:
        # Ajustar el tamaño y la posición del pájaro
        bird_resized = cv2.resize(bird_image, (50, 50))
        bird_alpha = bird_resized[:, :, 3] / 255.0
        bird_resized = bird_resized[:, :, :3]

        for c in range(0, 3):
            image[last_y_nose - 25:last_y_nose + 25, last_x_nose - 25:last_x_nose + 25, c] = \
                bird_resized[:, :, c] * (bird_alpha) + image[last_y_nose - 25:last_y_nose + 25, last_x_nose - 25:last_x_nose + 25, c] * (1 - bird_alpha)

    current_time = time.time()

    # Crear un nuevo tubo después de 3 segundos
    if current_time - inicio_juego > 3:
        if len(tubos) == 0 or tubos[-1][0] < (image.shape[1] - 300):
            tubos.append(crear_tubo(image.shape[1]))

    collision = actualizar_tubos(image, (last_x_nose, last_y_nose))
    if collision:
        break  # Salir del bucle si hay colisión

    actualizar_item(image, (last_x_nose, last_y_nose), current_time)

    # Dibujar puntaje
    cv2.putText(image, f'Puntaje: {puntaje}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow('MediaPipe Face Detection with Bird Overlay', image)
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

print("Game Over. Tu puntaje fue:", puntaje)



Game Over. Tu puntaje fue: 7
